# Simulating an Interchange with OpenMM

<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>
    The simplest way to install dependencies is to use the Interchange examples environment. From the root of the cloned openff-interchange repository:
    
    conda env create --name interchange-examples --file devtools/conda-envs/examples_env.yaml 
    conda activate interchange-examples
    pip install -e .
    cd examples/openmm
    jupyter notebook protein_ligand.ipynb
    
</details>

In this example, we'll quickly construct an `Interchange` and then run a simulation in OpenMM. 

We need an `Interchange` to get started, so let's put that together quickly. For more explanation on this process, take a look at the [packed_box] and [protein_ligand] examples.

[packed_box]: https://github.com/openforcefield/openff-interchange/tree/main/examples/packed_box
[protein_ligand]: https://github.com/openforcefield/openff-interchange/tree/main/examples/protein_ligand

In [ ]:
import time

from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.utils import get_data_file_path
from openff.units import unit
from openmm.app import PDBFile
import openmm
import nglview
import mdtraj as md
from pandas import read_csv

from openff.interchange import Interchange

# Read a structure from the Toolkit's test suite into a Topology
pdbfile = PDBFile(
    get_data_file_path("systems/packmol_boxes/propane_methane_butanol_0.2_0.3_0.5.pdb")
)
molecules = [Molecule.from_smiles(smi) for smi in ["CCC", "C", "CCCCO"]]
off_topology = Topology.from_openmm(pdbfile.topology, unique_molecules=molecules)

# Construct the Interchange with the OpenFF "Sage" force field
interchange = Interchange.from_smirnoff(
    force_field=ForceField("openff-2.0.0.offxml"), 
    topology=off_topology,
)
interchange.positions = pdbfile.positions

Tada! A beautiful solvent system:

In [ ]:
interchange.visualize("nglview")


## Run a simulation

We need OpenMM `System` and `Topology` objects to run our simulation, as well as positions, so lets prepare them first. We could just reuse the positions from the PDBFile and not have to worry about the units here, but in case you got your positions from somewhere else here's how to do it in the general case:

In [ ]:
openmm_sys = interchange.to_openmm(combine_nonbonded_forces=True)
openmm_top = interchange.topology.to_openmm()
openmm_pos = interchange.positions.m_as(unit.nanometer) * openmm.unit.nanometer

Let's choose parameters for the simulation and use them to prepare an Integrator:

In [ ]:
# Length of the simulation.
num_steps = 1000  # number of integration steps to run

# Logging options.
trj_freq = 10  # number of steps per written trajectory frame
data_freq = 10  # number of steps per written simulation statistics

# Integration options
time_step = 2 * openmm.unit.femtoseconds  # simulation timestep
temperature = 300 * openmm.unit.kelvin  # simulation temperature
friction = 1 / openmm.unit.picosecond  # friction constant

integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

Put the parts together and specify initial conditions:

In [ ]:
simulation = openmm.app.Simulation(openmm_top, openmm_sys, integrator)
simulation.context.setPositions(openmm_pos)
simulation.context.setVelocitiesToTemperature(temperature)

Configure how the simulation is recorded:

In [ ]:
# Configure the information in the output files.
pdb_reporter = openmm.app.PDBReporter("trajectory.pdb", trj_freq)
state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    data_freq,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

We're using a PDB reporter for simplicity but you should use something more space-efficient in production. Finally, run it!

In [ ]:
print("Starting simulation")
start = time.process_time()

# Run the simulation
simulation.step(num_steps)

end = time.process_time()
print(f"Elapsed time {end - start} seconds")
print("Done!")

We can take visualize the trajectory with NGLView:

In [ ]:
nglview.show_mdtraj(md.load("trajectory.pdb"))

And read the produced data with Pandas:

In [ ]:
read_csv("data.csv")